# Table Tools
## Part 2

In this notebook we will learn:

- About lists
- The `with_row` and `with_rows` functions
- The `group` and `pivot` functions, and the differences between them
- The use of statistical functions with group and pivot to compute stats by groups

In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

import scipy.stats as stats

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=np.VisibleDeprecationWarning)


## Lists

In [ ]:
simple_list = ['hello', 7, 3.14, True]
simple_list

In [ ]:
my_array = make_array(1, 2, 3)

crowded_list = [my_array, 'What is going on?', -10, [1,2,3], ["LCHS", "Lanier Tech", "UNG"]]
crowded_list

### Rows of Tables *are* Lists ###

Remember, columns of tables are arrays.  Arrays require that everything in them be of the same data type.  Lists don't have that requirement, which is good because a *row* of a table very likely should have different data types, whereas each individual column should be of only one type (most of the time).  

In [ ]:
Table().with_columns('Numbers', make_array(1, 2, 3))

In [ ]:
drinks = Table(['Drink', 'Cafe', 'Price'])
drinks

In [ ]:
drinks = drinks.with_rows([
    ['Milk Tea', 'Asha', 5.5],
    ['Espresso', 'Strada',  1.75],
    ['Latte',    'Strada',  3.25],
    ['Espresso', "FSM",   2]
])
drinks

In [ ]:
drinks = drinks.with_rows([["Coffee Frappuccino", "Starbucks", 4.45]])
drinks

## Grouping by one column

In [ ]:
survey = Table.read_table('welcome_survey_v4.csv')
survey.show(5)

In [ ]:
survey.group('Sleep position')

In [ ]:
survey.group("Sleep position").sort("count", descending=True).barh("Sleep position")

In [ ]:
survey.group('Sleep position', np.mean)

In [ ]:
help(np.average)

In [ ]:
survey.select('Sleep position', 'Hours of sleep').group('Sleep position', np.average)

In [ ]:
survey.select('Sleep position', 'Hours of sleep').group('Sleep position', np.average).barh("Sleep position")

In [ ]:
survey.select('Year', 'Hours of sleep').group('Year', np.average).sort("Hours of sleep average")

In [ ]:
survey.select('Year', 'Hours of sleep').group('Year', np.mean).show()
survey.select('Year', 'Hours of sleep').group('Year', np.mean).barh("Year")

## Cross-classification: grouping by two columns

In [ ]:
survey.group(['Handedness','Sleep position'], np.mean).show()

In [ ]:
survey.drop("Year").group(['Handedness','Sleep position'], np.mean).show()

In [ ]:
make_array(12, 2, 3)

In [ ]:
survey.select('Handedness','Sleep position', "Hours of sleep").group(['Handedness','Sleep position']).sort("count").show()

In [ ]:
survey.pivot('Sleep position', 'Handedness')

In [ ]:
survey.pivot('Sleep position', 'Handedness', 
             values='Hours of sleep', collect = np.mean)

In [ ]:
#help(Table.pivot)

In [ ]:
(survey.select('Handedness', 'Sleep position', 'Hours of sleep').group(['Handedness','Sleep position'], np.average))

In [ ]:
# Here, pivot doesn't know how to combine all the hours of sleep
# for each subgroup of students
#survey.pivot('Sleep position', 'Handedness', 'Hours of sleep')

## Challenge Questions ##

1. For each city, what’s the tallest building for each material?

2. For each city, what’s the height difference between the tallest steel building and the tallest concrete building?

In [ ]:
sky = Table.read_table('skyscrapers_v2.csv')
sky = (sky.with_column('age', 2022 - sky.column('completed'))
          .drop('completed'))
sky.show(3)

In [ ]:
sky.select('city', 'material', 'height').group(['city', 'material'], max).show()

In [ ]:
# 1. For each city, what’s the tallest building for each material?

sky.pivot('material', 'city', 'height', max).show()

#sky.select('city','material','height').group(['city', 'material' ], max).show()



In [ ]:
# 2. For each city, what’s the height difference between the tallest 
#    steel building and the tallest concrete building?

mytable = sky.pivot('material','city', 'height', max)

mytable = mytable.with_column('Diff', abs(mytable.column(3)-mytable.column(1)))
mytable

In [ ]:
mytable1 = sky.pivot('material','city', 'height', max).drop('concrete','mixed/composite')
mytable1

mytable2 = sky.pivot('material','city', 'height', min).drop('concrete','mixed/composite')
#mytable2

mytable3 = mytable1.with_column("Difference", mytable1.column(1)-mytable2.column(1)).drop('steel')
mytable3.show()

In [ ]:
def my_diff(x, y, z):
    return max(x, y, z) - min(x, y, z)

mytable = sky.pivot('material','city', 'height', max)

mytable = mytable.with_column("New Diff", mytable.apply(my_diff, "concrete", "mixed/composite","steel"))
mytable

Don't read ahead until you try the challenge questions yourself first!

In [ ]:
sky.select('material', 'city', 'height').group(['city', 'material'], max)

In [ ]:
sky_p = sky.pivot('material', 'city', values='height', collect=max)
sky_p.show()

In [ ]:
sky_p = sky_p.with_column(
    'difference', 
    abs(sky_p.column('steel') - sky_p.column('concrete'))
)
sky_p

In [ ]:
sky_p.sort('difference', True)

In [ ]:
# Generate a table of the names of the oldest buildings for each 
# material for each city:

def ident(x):
    return x.item(0)

sky.group(["city", "material"], max).pivot("material", "city", "name max", ident)


In [ ]:
sky.pivot("material", "city", "age", max)

In [ ]:
sky.group(["city", "material"], max)

In [ ]:
"DOg"<"Dog"<"dog" <"dogs"

In [ ]:
np.arange(1,5,2)

In [ ]:
drinks.take(np.arange(2))

## Billionaires ##

In [ ]:
billionaires = Table.read_table('billionaires2.csv')
billionaires

In [ ]:
(billionaires.group('citizenship')
             .sort(1, descending=True)
             .barh('citizenship'))

In [ ]:
billionaires.group('industry').sort(1, descending=True)

In [ ]:
billionaires.group('industry').sort(1, descending=True).barh("industry")

In [ ]:
billionaires.pivot('industry', 'citizenship').show(5)

In [ ]:
countries = Table.read_table('countries.csv')
countries

In [ ]:
countries.where('Continent', 'Oceania').show()

In [ ]:
billionaires.pivot('industry', 'Continent')

In [ ]:
ceos = Table.read_table('ceo.csv')
ceos

In [ ]:
ceos.hist('amount')

In [ ]:
ceos.sort('amount', descending=True)

In [ ]:
ceos.sort('amount', descending=False)

In [ ]:
ceos.hist('amount', bins=np.arange(0, 1e5, 1e3), unit='$1000');

In [ ]:
ceos.hist('amount', bins=np.arange(0, 1e5, 1e3), unit='$1000')

In [ ]:
b_donations = Table.read_table("billionaire donations.csv")

b_donations.where('amount', are.below(6e6)).scatter('net worth', 'amount')

In [ ]:
donation_prop = 1000 * b_donations.column('amount') / b_donations.column('net worth')
b_donations = b_donations.with_column('Contribution%', donation_prop)
b_donations = b_donations.sort('amount', descending=True)
b_donations.set_format('Contribution%', PercentFormatter)
b_donations.show(20)

In [ ]:
b_donations.sort('name', descending=True).show()

In [ ]:
6.2e9/38e9